<a href="https://colab.research.google.com/github/Sidy3143/llm-projects/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q fsspec==2025.3.0 gcsfs transformers accelerate peft bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q wandb

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sidyn7710 (sidyn7710-sidy) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
#from torch.utils.tensorboard import SummaryWriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
model_name = "gpt2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"":0}
)

model.config.use_cache = False #no kv cache
model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = "right"
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
print(f"Model loaded: {model_name}")
print(f"Model parameters: {model.num_parameters():,}")

Model loaded: gpt2
Model parameters: 124,439,808


In [5]:
from datasets import load_dataset

ds = load_dataset("yahma/alpaca-cleaned")

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
ds['train'][0] #['instruction']

{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'input': '',
 'instruction': 'Give three tips for staying healthy.'}

In [6]:
def process(example):
  instruction = example['instruction']
  input = example['input']
  output = example['output']

  if input.strip():
    prompt = (f"### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{output}")
  else:
    prompt = (f"### Instruction:\n{instruction}\n\n### Response:\n{output}")

  return {'text': prompt}

formatted_data = ds.map(process, remove_columns=['input', 'output', 'instruction'])

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
formatted_data['train'][0]

{'text': '### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'}

In [7]:
def tokenize_function(example):
  tokenized = tokenizer(example['text'],
                        truncation=True,
                        max_length=512,)

  return tokenized

tokenized_dataset = formatted_data.map(tokenize_function, batched=True, remove_columns=['text']) #,

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 51760
    })
})

In [8]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType, PeftModel

In [9]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj", "c_fc"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM, # or "CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8607


In [10]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from trl import SFTTrainer

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/gpt2-chat",

    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size =4,

    gradient_accumulation_steps=4,
    gradient_checkpointing=True,

    fp16=False,
    bf16=True,  # Use mixed precision training

    dataloader_pin_memory=True,

    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",

    max_grad_norm=0.3,
    weight_decay=0.001,
    warmup_ratio=0.03,
    learning_rate=2e-4,

    group_by_length=True,

    logging_steps=100,
    save_steps=1000,
    eval_steps=1000,

    remove_unused_columns=False,

    report_to="wandb",
    load_best_model_at_end=False,
)

In [ ]:
split_dataset = tokenized_dataset['train'].train_test_split(test_size=0.1, seed=43)
train_data = split_dataset['train']
eval_data = split_dataset['test']

In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 46584
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5176
    })
})

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    peft_config=lora_config,
    train_dataset=train_data,
    eval_dataset=eval_data,
)

Truncating train dataset:   0%|          | 0/46584 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/5176 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print("\n=== Starting Training ===")
checkpoint_path = "/content/drive/MyDrive/gpt2-chat/checkpoint-2000" # for resuming
trainer.train(resume_from_checkpoint = checkpoint_path)


=== Starting Training ===


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
2100,2.078600
2200,2.052400
2300,2.058900
2400,2.048900
2500,2.053900
2600,2.041600
2700,2.042800
2800,2.065100
2900,2.037600
3000,2.081700


TrainOutput(global_step=5823, training_loss=1.3310777126012872, metrics={'train_runtime': 2506.5698, 'train_samples_per_second': 37.17, 'train_steps_per_second': 2.323, 'total_flos': 9139313099390976.0, 'train_loss': 1.3310777126012872})

In [ ]:
trainer.save_model(checkpoint_path)

In [ ]:
from peft import PeftModel

In [ ]:
#freeze the memory and reload the trained model
del model
del trainer

In [ ]:
model_path = "/content/drive/MyDrive/gpt2-chat"
tokenizer_path = "/content/drive/MyDrive/gpt2-chat"

# Load the base model first (gpt2 in this case)
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

model = PeftModel.from_pretrained(base_model, model_path)

# Ensure the model is in evaluation mode
model.eval()

# Move the model to the CUDA device
import torch
device = torch.device("cuda")
model.to(device)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
        

In [11]:
instruction = "Write a short story about a robot who learns to love."
prompt = (f"### Instruction:\n{instruction}\n\n### Response:")

In [13]:
import torch
device = torch.device("cuda")

In [15]:
input_ids = tokenizer(prompt, return_tensors="pt")
input_ids = input_ids.to(device)

In [ ]:
output = model.generate(input_ids['input_ids'], max_length=200,)

# Decode the generated output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Instruction:
Write a short story about a robot who learns to love.

### Response:
As a young boy, I was fascinated by the beauty and wonder of robots. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their successes. I loved the way they could learn from their mistakes and learn from their succes